In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as py
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from functools import partial, reduce
from itertools import combinations
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore
%matplotlib inline 

In [2]:
class Rfilter(object):

    def __init__(self, hadron='pi+', fudge=[0, 0]):
        self.M = 0.938
        self.M2 = self.M**2
        self.set_Mh(hadron)
        

        self.fudge1 = fudge[0]
        self.fudge2 = fudge[1]

        self.MiT2 = 0.5 + 0.3 * fudge[0]
        self.MfT2 = 0.5 + 0.3 * fudge[1]
        self.kT = 0
    
        self.MX = 1.3
        self.Ma = 1.5
        self.Mb = 0.3
        self.deltaM = 0.3
        self.MJ = 0.3
        
        self.MiT = self.MiT2**0.5
        self.MfT = self.MfT2**0.5
        self.kT2 = self.kT**2
        
        # from paper, we need kf2, deltakT2, ki2
        # we need to have a way to vary them!
        self.kf       = 0
        self.ki       = 0
        self.kf2      = self.kf**2
        self.ki2      = self.ki**2
        self.deltakT  = 0
        self.deltakT2 = self.deltakT**2
    def set_ki(self,ki):
        self.ki = ki
        self.ki2 = ki**2
    def set_kf(self,kf):
        self.kf = kf
        self.kf2 = kf**2
    def set_kT(self,kT):
        self.kT = kT
        self.kT2 = kT**2
    def set_deltakT(self,deltakT):
        self.deltakT = deltakT
        self.deltakT2 = self.deltakT**2
    def set_Mh(self, hadron):
        if hadron == 'pi+':
            self.Mh = 0.135  #  
        if hadron == 'pi-':
            self.Mh = 0.135  #  
        if hadron == 'pi0':
            self.Mh = 0.135  #  
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'k-':
            self.Mh = 0.493
        if hadron == 'k0':
            self.Mh = 0.493
        if hadron == 'h+':
            self.Mh = 
        self.Mh2 = self.Mh**2
    
    def get_ki2(self):
        return self.ki2
    def get_kf(self):
        return self.kf
    
    def get_W2(self, x, Q2):
        W2 = Q2 * (1. - x) / x + self.M2
        return W2

    def get_MBT(self, PhT):
        MBT = np.sqrt( self.Mh2 + PhT**2 )
        return MBT
        
    def get_MiT(self, x, Q2):
        xn = self.get_xn(x, Q2)
#    self.MiT = np.sqrt( (xn*self.kT**2 + xn*self.MX**2 - (1-xn)*xn*self.M2)/(1-xn) )
        self.MiT = np.sqrt((xn * self.kT**2 + xn * (self.Ma + self.Mb / np.sqrt(xn))
                            ** 2 - (1 - xn) * xn * self.M2 + self.deltaM**2 * self.fudge1) / (1 - xn))
        return self.MiT

    def get_MfT(self):
        self.MfT = np.sqrt(self.kT**2 + self.MJ**2 +
                           self.deltaM**2 * self.fudge2)
        return self.MfT

    def get_xn(self, x, Q2):
        return 2 * x / (1 + np.sqrt(1 + 4 * x**2 * self.M2 / Q2))

    def get_yh(self, x, z, Q2, PhT, hadron, sign=-1):
        xn = self.get_xn(x, Q2)
        self.set_Mh(hadron)
        expy = Q2**0.5 * z * (Q2 - xn**2 * self.M2)\
            / (2 * self.M2 * xn**2 * (self.Mh2 + PhT**2)**0.5)\
            + sign * Q2**0.5 / (xn * self.M) * (z**2 * (Q2 - xn**2 * self.M2)**2
                                        / (4 * self.M2 * xn**2 * (self.Mh2 + PhT**2)) - 1)**0.5
        return np.log(expy)


    # zn
    def get_zn(self, x, z, Q2, PhT, hadron):
        xn = self.get_xn(x,Q2)
        self.set_Mh(hadron)
        MBT = self.get_MBT(PhT)
        zn =  xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        return zn
        
# rapidity of the target

    def get_yp(self, x, Q2):
        xn = self.get_xn(x, Q2)
        return np.log(np.sqrt(Q2) / (xn * self.M))

    def get_yi(self, Q2):
        return 0.5 * np.log(Q2 / self.MiT**2)

    def get_yf(self, Q2):
        return -0.5 * np.log(Q2 / self.MfT**2)

    def get_MhT(self, PhT):
        return np.sqrt(self.Mh2 + PhT**2)

    def get_R(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        MfT = self.get_MfT()
        MiT = self.get_MiT(x, Q2)
        yi = self.get_yi(Q2)
        yf = self.get_yf(Q2)
        MhT = self.get_MhT(PhT)
        yh = self.get_yh(x, z, Q2, PhT, hadron)
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        znhat = zn/z
        # from paper..
        Ph_kf = 0.5 * MhT * MfT * (np.exp(yf - yh) + np.exp(yh - yf)) - znhat/zn * PhT**2 - PhT * self.kT
        Ph_ki = 0.5 * MhT * MiT * (np.exp(yi - yh) - np.exp(yh - yi)) - PhT * self.kT
        return np.abs(Ph_kf / Ph_ki)

        
# R0, Eq. (4.14)...
    def get_R0(self, Q2):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  np.maximum(np.maximum(self.ki2/Q2,self.kf2/Q2),self.kT2/Q2)
        
        
# We call R1 in the new paper what was R in the previous...
    def get_R1(self, x, z, Q2, PhT, hadron):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  self.get_R( x, z, Q2, PhT, hadron)  

# R2 from Eq. (4.17)        
    def get_R2(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        znhat = zn/z
        qT = -PhT/zn
        # from paper..
        return np.abs( -(1.-znhat) - znhat*qT**2/Q2 - (1.-znhat)*self.kf2/(znhat*Q2) - \
                      self.deltakT2/(znhat*Q2) + 2.*qT*self.deltakT/Q2 )

# Proximity is defined for a sphere of a selected radius(p) by the following function
def Proximity(files,path,domain,p,fudge):
    """
    data   = any Pandas DataFrame should do as long as it contains x,z,Q2, and pT
    domain = an array of values you wish to extract R0,R1,R2 
    p      = radius of sphere representing the demarcation of the proximity
    hadron = any hadron found in the data
    fudge  = idk what fudge is so I did the right thing and passed it along 
    """
    proximity = {}
    for file in files:
        data = pd.read_excel(path + file)
        #print(len(data))
       #proximity.update() = file
        for i in range(0,len(data)):
            hadron = data.loc[i,'hadron']
            #print(hadron)
            #print(file)
            Rval = Rfilter(hadron = hadron,fudge=fudge)
            inside = 0.
            outside = 0.
            
            for ki in domain:
                Rval.set_ki(ki)
                for kf in domain:
                    Rval.set_kf(kf)
                    for kT in domain:
                        Rval.set_kT(kT)
                        Rval.set_deltakT(kT)
            
                        R0 = Rval.get_R0(data.loc[i,'Q2'])
                        R1 = Rval.get_R1(data.loc[i,'x'],data.loc[i,'z'],data.loc[i,'Q2'],data.loc[i,'pT'], \
                                     hadron = hadron)
                        R2 = Rval.get_R2(data.loc[i,'x'],data.loc[i,'z'],data.loc[i,'Q2'],data.loc[i,'pT'], \
                                     hadron = hadron)
                        p2 = R0**2 + R1**2+ R2**2
                    
                        if p2 < 1:
                            inside += 1.
                        
                        else:
                            outside += 1.
            if i == 0:
                proximity.update({file : {i : inside/(inside+outside)}})
            else:
                
                proximity[file].update( { i : inside/(inside+outside)})
        
    return proximity

In [5]:
#files = ['1000.xlsx','1001.xlsx','1002.xlsx','1003.xlsx','1004.xlsx','1005.xlsx',
#         '1006.xlsx','1007.xlsx','1008.xlsx','1009.xlsx']
files = ['1000.xlsx','1001.xlsx','1002.xlsx','1003.xlsx','1004.xlsx','1005.xlsx',
         '1006.xlsx','1007.xlsx']
path = '~/GIT/collinearity/database/sidis/expdata/'
domain = np.linspace(.2,.8,10)
p = 1
fudge = [0,0]
proximity = Proximity(files,path,domain,p,fudge)

In [6]:
proximity

{'1000.xlsx': {0: 0.3333333333333333,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.6666666666666666,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.6666666666666666,
  15: 0.14814814814814814,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 0.6666666666666666,
  22: 0.3333333333333333,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.6666666666666666,
  29: 0.48148148148148145,
  30: 0.037037037037037035,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 0.6666666666666666,
  36: 0.6666666666666666,
  37: 0.3333333333333333,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.0,
  42: 0.6666666666666666,
  43: 0.6666666666666666,
  44: 0.6666666666666666,
  45: 0.3333333333333333,
  46: 0.0,
  47: 0.0,
  48: 0.0,
  49: 1.0,
  50: 0.6666666666666666,
  51: 0.6666666666666666,
  52: 0.6666666666666666,
  53: 0.3333333333333333,
  54: 0.0,
  55: 0.0,
  56: 0.3333333333333333,
  57: 0.0,
  58: 0.0,
  59: 0.0,
  60: 0.0,
  61: 0

In [9]:
proximity.keys()

dict_keys(['1000.xlsx', '1001.xlsx', '1002.xlsx', '1003.xlsx', '1004.xlsx', '1005.xlsx', '1006.xlsx', '1007.xlsx'])

In [10]:
proximity['1000.xlsx'].keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,